In [1]:
import requests
import re

from bs4 import BeautifulSoup as bs

import time
import random

import pandas as pd

import numpy as np

import os

In [2]:
years = range(2018, 2024)

In [3]:
team_names=[
    '49ers',
    'Cardinals',
    'Falcons',
    'Ravens',
    'Buffalos',
    'Panthers',
    'Bears',
    'Bengals',
    'Browns',
    'Cowboys',
    'Broncos',
    'Lions',
    'Packers',
    'Texans',
    'Colts',
    'Jaguars',
    'Chiefs',
    'Raiders',
    'Chargers',
    'Rams',
    'Dolphins',
    'Vikings',
    'Patriots',
    'Saints',
    'Giants',
    'Jets',
    'Eagles',
    'Steelers',
    'Seahawks',
    'Buccaneers',
    'Titans',
    'Commanders'
]

In [4]:
team_names.sort()

In [5]:
def get_player_li (team: str, year: int):

    roster = pd.read_csv('NFL_Data_{}/{}/{}_roster'.format(year, team, team))

    roster = roster[:-1]

    #reset index
    roster.reset_index(drop=['index'], inplace=True)

    #get list of players
    players_li = list(roster['Player'].values)[:-1]
    
    return players_li

In [6]:
#loop through player_name list to get each player url
#url_number to differentiates similar players similar url names
#year to select which season

### this uses the first 2 characters of the first name
def player_url_func_1(player: str, url_number: int, year: int):
    
    '''
    player = str search for the player you're looking for
    url_number = int the number to distinguish between similar player names
    year = int which year you're searching in
    '''
    
    
    #split the names
    if player.isalpha() == False:
        
        #delete ' from names
        name_replace = player.replace("'", "")

        #leave out anything other than names
        split_name = name_replace.split()[:2]
            
            
    else:
        split_name = player.split()

    #players last name first character
    player_ln_1c = split_name[1][0]
    
    #players first two characters of first name
    player_fn_2c = split_name[0][:2]
        
    player_fn_2c.strip()
        
    #first four characters of last name
    if len(split_name[1]) < 4:
        player_ln_4c = split_name[1]+'x'
        
    else:
        player_ln_4c = split_name[1][:4]


    #the url
    #in order, the brackets to fill in
        #fist character from last name
        #first four characters from last name
        #first two characters of first name
        # number code for to differentiate similar url because it is based off names/initial
        #+1 when name doesn't match
        #year
    url = 'http://www.pro-football-reference.com/players/{}/{}{}{}/gamelog/{}/advanced/'

    #for the url code to differentiate names
    url_no = '{:02d}'.format(url_number)

    #url code
    url_requests = url.format(player_ln_1c, player_ln_4c, player_fn_2c, url_no, year)

    return url_requests

In [7]:
#loop through player_name list to get each player url
#url_number to differentiates similar players similar url names
#year to select which season

### this uses the first 3 characters of the first name
def player_url_func_2(player: str, url_number: int, year: int):
   
    '''
    Some url use a different number of characters from the player's name, to create up the url. 
    
    player = str search for the player you're looking for
    url_number = int the number to distinguish between similar player names
    year = int which year you're searching in
    '''
    
    #split the names
    if player.isalpha() == False:

        #delete ' from names
        name_replace = player.replace("'", "")

        #leave out anything other than names
        split_name = name_replace.split()[:2]

    else:
        split_name = player.split()

    #players last name first character
    player_ln_1c = split_name[1][0]

    #players first two characters of first name
    player_fn_3c = split_name[0][:3]


    #first four characters of last name
    if len(split_name[1]) < 4:
        player_ln_4c = split_name[1]+'xx'
    else:
        player_ln_4c = split_name[1][:5]


    #the url
    #in order, the brackets to fill in
        #fist character from last name
        #first four characters from last name
        #first two characters of first name
        # number code for to differentiate similar url because it is based off names/initial
            #+1 when name doesn't match
        #year
    url = 'http://www.pro-football-reference.com/players/{}/{}{}{}/gamelog/{}/advanced/'

    #for the url code to differentiate names
    url_no = '{:02d}'.format(url_number)

    #url code
    url_requests = url.format(player_ln_1c, player_ln_4c, player_fn_3c, url_no, year)

    return url_requests

In [8]:
def webscrape_player_name(url: str):
    '''
    Webscrape the for the player's name by generated url.
    
    url = str webscrapes url for player's name
    '''
    
    #request html
    html = requests.get(url).text

    time.sleep(random.randrange(5,7))

    #parse html
    soup = bs(html, 'html.parser')

    #find header for player name
    h1 = soup.find_all('h1')[0]


    try:
        #get player name
        span = h1.find_all('span')[0]

        name = span.contents[0]

    except:
        name = False


    return name

In [9]:
def get_player_url_1(search_player:str, year:int):
    
    '''
    Search for the correct player from the roster. 
    
    search_player = str Player you are searching for
    year = int The year of the roster youre using
    '''

    url_number_li = [0, 1, 2, 3, 4, 5, 6, 7, 20, 21, 44, 99, 'n/a']
    
    
    for url_number in url_number_li:
        try:
            
            #url for initial search for player name on url
            search_player_url = player_url_func_1(player=search_player, url_number=url_number, year=year)
            
            player1 = webscrape_player_name(url=search_player_url)
            
            # matcing which
            if player1 != search_player:
                pass
            
            else:
                break
        
        except:
            return 'n/a_' + search_player
        
    return search_player_url

In [10]:
def get_player_url_2(search_player:str, year:int):
    url_number= 1
    
    try:
        #url for initial search for player name on url
        search_player_url = player_url_func_2(player=search_player, url_number=url_number, year=year)
            
        #player scraped from url    
        player1 = webscrape_player_name(url=search_player_url)
            
            # matcing which
        if player1 != search_player:
            return 'n/a_' + search_player
        
    except:
        return 'n/a_' + search_player
        

    return search_player_url

In [11]:
# get final and correct url
##### maybe###########
def final_url(player: str, year: int):
    '''
    Connecting all the functions. 
    '''
    try:
        get_player_url = get_player_url_1(search_player=player, year=year)
        
        time.sleep(random.randrange(5,7))

        if get_player_url == 'n/a_'+player:
            get_player_url = get_player_url_2(search_player=player, year=year)

        
    except:
        return 'n/a_' + search_player

        
        
    return get_player_url

In [12]:
def webscrape_player_adv_log_table(player_url: str, team: str, year: int):
    '''
    url_li = teams roster url list in string
    '''
    try:
        #scrape table
        df_tables = pd.read_html(player_url)
        
        time.sleep(random.randrange(5,7))
        
        if len(df_tables) > 1:
            #numer to label different tables fromw same player
            no = 0
            for table in df_tables:
                    
                df = table.droplevel(level=0, axis=1)

                #get name
                name = webscrape_player_name(player_url)

                #to csv
                #to local file
                df.to_csv('NFL_Data_{}/{}/Players Log/{}_{}'.format(year, team, name, no))
                    
                no+=1
                    
        else:
            df = df_tables[0].droplevel(level=0, axis=1)

            #get name
            name = webscrape_player_name(player_url)

            #to csv
            df.to_csv('NFL_Data_{}/{}/Players Log/{}'.format(year, team, name))

    except:
        pass
        
    return 

In [13]:
team_names

['49ers',
 'Bears',
 'Bengals',
 'Broncos',
 'Browns',
 'Buccaneers',
 'Buffalos',
 'Cardinals',
 'Chargers',
 'Chiefs',
 'Colts',
 'Commanders',
 'Cowboys',
 'Dolphins',
 'Eagles',
 'Falcons',
 'Giants',
 'Jaguars',
 'Jets',
 'Lions',
 'Packers',
 'Panthers',
 'Patriots',
 'Raiders',
 'Rams',
 'Ravens',
 'Saints',
 'Seahawks',
 'Steelers',
 'Texans',
 'Titans',
 'Vikings']

In [18]:
# 2018 browns 12/6/2023
year=2018
team = 'Chargers'

In [15]:
asd

NameError: name 'asd' is not defined

In [16]:
players_li = get_player_li(team=team_names[-1], year=year)
players_li

['Ameer Abdullah',
 'Mackensie Alexander',
 'Dan Bailey',
 'Anthony Barr',
 'Chad Beebe',
 'Mike Boone',
 'Tashawn Bower',
 'Kentrell Brothers',
 'Daniel Carlson',
 'Stacy Coley',
 'Tom Compton',
 'Tyler Conklin',
 'Dalvin Cook',
 'Kirk Cousins',
 'Stefon Diggs',
 'Devante Downs',
 'Pat Elflein',
 'Ben Gedeon',
 'Everson Griffen',
 'C.J. Ham',
 'Anthony Harris',
 'Holton Hill',
 'Rashod Hill',
 'Jalyn Holmes',
 'Mike Hughes',
 'Danielle Hunter',
 'George Iloka',
 'Danny Isidora',
 'Craig James',
 'Jaleel Johnson',
 'Tom Johnson',
 'Brett Jones',
 'Linval Joseph',
 'Jayron Kearse',
 'Eric Kendricks',
 'Kevin McDermott',
 'David Morgan',
 'Latavius Murray',
 'Storm Norton',
 "Brian O'Neill",
 'David Parry',
 'Riley Reiff',
 'Mike Remmers',
 'Xavier Rhodes',
 'Sheldon Richardson',
 'Aldrick Robinson',
 'Kyle Rudolph',
 'Andrew Sendejo',
 'Marcus Sherels',
 'Harrison Smith',
 'Adam Thielen',
 'Roc Thomas',
 'Laquon Treadwell',
 'Trae Waynes',
 'Stephen Weatherly',
 'Matt Wile',
 'Eric Wils

In [ ]:
#2018 player logs3
n_a_li = []
for team in team_names:
    print(team)
    players_li = get_player_li(team=team, year=year)
    for player in players_li:    
        adv_log_player_url = final_url(player=player, year=year)
        webscrape_player_adv_log_table(player_url=adv_log_player_url, team=team, year=year)

        print(adv_log_player_url)
        
        if adv_log_player_url[:3] == 'n/a':
            n_a_li.append(adv_log_player_url)

49ers
http://www.pro-football-reference.com/players/A/ArmsAr01/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BeatC.00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BlaiRo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BoldVi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BourKe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BreiMa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BuckDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/CeleGa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/ColbAd00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/CoylBr00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/D/DayxSh00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/D/DayeMa00/gamelog/2018/advanced/
http://www.pro-football-reference.

http://www.pro-football-reference.com/players/M/MizzTa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/N/NichBi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/O/ODonPa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/P/ParkCo03/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/R/RobeRo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/R/RobiAl02/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/S/ScalPa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/S/ShahAd00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/S/SimsDi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/S/SmitRo07/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/S/SoweBr00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/T/ToliKe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/pl

C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a_Jordan Willis
http://www.pro-football-reference.com/players/W/WilsBr01/gamelog/2018/advanced/
Broncos
http://www.pro-football-reference.com/players/B/BarrSh00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BierKe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BollGa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BookDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BrocTr00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/ButtJa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/ChubBr00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/CracRi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/C/CravSu00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/D/DaviTo01/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/F/FreeRo00/gamelog/2018/advanced/
http://www.pro

C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


...

n/a_Bobo Wilson
http://www.pro-football-reference.com/players/W/WilsSh01/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WinsJa00/gamelog/2018/advanced/
Buffalos
http://www.pro-football-reference.com/players/A/AlexLo99/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/A/AlleJo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/A/AndeDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BarkMa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BenjKe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BodiRu00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BoetIk00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BojoCo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BoldVi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BushRa00/gamelog/2018/advanced/
http://www.pro-

http://www.pro-football-reference.com/players/G/GoldMa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/G/GoldZa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/G/GonzZa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/G/GossCo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/G/GresJe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/G/GuntRo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/H/HodgGe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/H/HoldWi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/H/HolmGa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/H/HumpD.00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/I/IupaMi20/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/J/JohnDa05/gamelog/2018/advanced/
http://www.pro-football-reference.com/pl

http://www.pro-football-reference.com/players/S/SquaDa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/S/SturCa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/T/TeviSa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/T/ToneCo00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WattDe00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WhitKy00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WillMi00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WillTr05/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WillTy00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WilsKy00/gamelog/2018/advanced/
Chiefs
http://www.pro-football-reference.com/players/A/AlleJe01/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/B/BailAl00/gamelog/2018/advanced/
http://www.pro-football-reference

C:\Users\Family\anaconda3\Lib\site-packages\pandas\io\html.py:674: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(udoc, features="html5lib", from_encoding=from_encoding)


n/a_Darrel Williams
http://www.pro-football-reference.com/players/W/WillXa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WincJa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/W/WyliAn00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/Y/YeldDe00/gamelog/2018/advanced/
Colts
http://www.pro-football-reference.com/players/A/AdamMa00/gamelog/2018/advanced/
http://www.pro-football-reference.com/players/A/AlieMo00/gamelog/2018/advanced/


In [ ]:
###    N/A List   ###

### 2018 
    #49ers
        #-
    #Bears
        #-
    #Bengals
        #n/a_Jordan Willis
    #Broncos
        #n/a_Adam Jones
    #Browns
        #n/a_T.J. Carrie, n/a_J.C. Tretter
    #Buccaneers
        #n/a_Adarius Taylor, n/a_Bobo Wilson
    #Buffalos
        #-
    #Cardinals
        # - 
    #chargers
        #-
    #Cheifs
        #
    #Colts
        #
    #commanders
        #
    #cowboys
        #
    #Dophins
        #
    #Eagles
        #
    #Giants 
        #
    #Jaguars
        #
    #Jets
        #
    #Lions
        #
    #Packers
        #
    #Panthers
        #
    #Patriots
        #
    #Ravens
        #
    #Rams
        #
    #Saints
        #
    #Seahawks
        #
    #Steelers
        #
    #Texans
        #
    #Titans
        #
    #Vikings
        #

In [ ]:
#get url into data frame

In [ ]:
#Graham Gano Panthers 2018
#https://www.pro-football-reference.com/players/G/GanoGr44.htm